# Unified MDTF/GFDL/NCAR Analysis Notebook Template

More details on the development process:
[MDTF Planning Document](https://docs.google.com/document/d/1P8HqL8O5304qwR3ik9RmgFDwSWwlkPgOjnp39PIkLfY/edit?usp=sharing)

In [ ]:
# Development mode: constantly refreshes module code
%load_ext autoreload
%autoreload 2

## Framework Code and Diagnostic Setup

In [ ]:
import os
os.environ["ESNB_LOG_LEVEL"] = "DEBUG"

In [ ]:
import esnb
from esnb import NotebookDiagnostic, RequestedVariable, CaseGroup2

import intake
import intake_esm

In [ ]:
# Define a mode (leave "prod" for now)
mode = "prod"

# Verbosity
verbose = True

# Give your diagnostic a name and a short description
diag_name = "Example Pangeo Notebook"
diag_desc = "Demonstration of loading Pangeo datasets"

# Define what variables you would like to analyze. The first entry is the
# variable name and the second entry is the realm (post-processing dir).
#   (By default, monthly timeseries data will be loaded. TODO: add documentation
#    on how to select different frequencies, multiple realms to search, etc.)
variables = [
    RequestedVariable("tas", "Amon"),
    RequestedVariable("prw", "Amon"),
]

# Optional: define runtime settings or options for your diagnostic
user_options = {"regrid_to_common": True}

# Initialize the diagnostic with its name, description, vars, and options
diag = NotebookDiagnostic(diag_name, diag_desc, variables=variables, **user_options)

In [ ]:
# Manually search the pangeo catalog for each case
pangeo_catalog = intake.open_esm_datastore(esnb.datasources.cmip6_pangeo)

mpi = pangeo_catalog.search(
    experiment_id="historical", 
    table_id="Amon", 
    source_id="MPI-ESM1-2-LR", 
    member_id="r1i1p1f1"
)

canesm5 = pangeo_catalog.search(
    experiment_id="historical", 
    table_id="Amon", 
    source_id="CanESM5", 
    member_id="r1i1p1f1",
)

In [ ]:
# Convert the panego-style catalogs to GFDL Catalog Builder format
mpi = esnb.core.util_catalog.convert_pangeo_catalog_to_catalogbuilder(mpi)
canesm5 = esnb.core.util_catalog.convert_pangeo_catalog_to_catalogbuilder(canesm5)

In [ ]:
groups = [
    CaseGroup2(mpi, date_range=("1981-01-01", "2010-12-31"), name="MPI-ESM1-2-LR"),
    CaseGroup2(canesm5, date_range=("1981-01-01", "2010-12-31"), name="CanESM5")
]

In [ ]:
# Combine the experiments with the diag request and determine what files need to be loaded:
diag.resolve(groups)

In [ ]:
# Print a list of file paths
# This cell and the markdown cell that follows are necessary to run this notebook
# Interactively on Dora
_ = [print(x) for x in diag.files]

<i>(The files above are necessary to run the diagnostic.)</i>

In [ ]:
# Load the data as xarray datasets
diag.open()

## Exploring Datasets by Looping Over Variables

In [ ]:
# First loop over variables, and then over groups

for variable in diag.variables:
    for group in variable.datasets.keys():
        ds = variable.datasets[group]
        print("\n")
        print(f"Variable={variable}, Group={group}")
        print(ds)

## Exploring Datasets by Looping over Groups

In [ ]:
# First loop over groups, and then over variables

for group in diag.groups:
    for variable in group.datasets.keys():
        ds = group.datasets[variable]
        print("\n")
        print(f"Variable={variable}, Group={group}")
        print(ds)